In [78]:
import pandas as pd
import re

### The screening question were already scanned and participants were filtered based on responses manually to remove unauthentic, erroneous and incomplete responses. Here we will directly proceed to the cleaning of data.

### Read csv

In [79]:
df = pd.read_csv("qualtrics_data.csv")

### Combine list of unused columns

In [80]:
columns_to_drop = [
    "StartDate",
    "EndDate",
    "Status",
    "IPAddress",
    "RecordedDate",
    "RecipientLastName",
    "RecipientFirstName",
    "RecipientEmail",
    "ExternalReference",
    "LocationLatitude",
    "LocationLongitude",
    "DistributionChannel",
    "UserLanguage",
    "consent",
    "combination",
    "priorExperience",
    "Q111",
    "Q24",
    "Q25",
]

### Drop unused columns

In [81]:
df = df.drop(columns_to_drop, axis=1)

### Drop duplicate ResponseId

In [82]:
df = df.drop(df.iloc[:, [66]], axis=1)

### Create list of columns for renaming

In [83]:
rename_columns = {
    "Q93_1":"t3_1",
    "Q93_2":"t3_2",
    "Q93_3":"t3_3",
    "Q93_4":"t3_4",
    "Q98_1":"t6_1",
    "Q98_2":"t6_2",
    "Q98_3":"t6_3",
    "Q98_4":"t6_4",
    "Q99_1":"t9_1",
    "Q99_2":"t9_2",
    "Q99_3":"t9_3",
    "Q99_4":"t9_4",
    "Q100_1":"t12_1",
    "Q100_2":"t12_2",
    "Q100_3":"t12_3",
    "Q100_4":"t12_4",
    "Q101_1":"t15_1",
    "Q101_2":"t15_2",
    "Q101_3":"t15_3",
    "Q101_4":"t15_4",
    "Q102_1":"t18_1",
    "Q102_2":"t18_2",
    "Q102_3":"t18_3",
    "Q102_4":"t18_4",
    "Q103_1":"t21_1",
    "Q103_2":"t21_2",
    "Q103_3":"t21_3",
    "Q103_4":"t21_4",
    "Q104_1":"t24_1",
    "Q104_2":"t24_2",
    "Q104_3":"t24_3",
    "Q104_4":"t24_4",
    "Q105_1":"t27_1",
    "Q105_2":"t27_2",
    "Q105_3":"t27_3",
    "Q105_4":"t27_4",
    "Q106_1":"t30_1",
    "Q106_2":"t30_2",
    "Q106_3":"t30_3",
    "Q106_4":"t30_4",
    "Q107_1":"t33_1",
    "Q107_2":"t33_2",
    "Q107_3":"t33_3",
    "Q107_4":"t33_4",
    "Q108_1": "t36_1",
    "Q108_2": "t36_2",
    "Q108_3": "t36_3",
    "Q108_4": "t36_4",
    "feature_usefullness_1" : "sidebar_rating",
    "feature_usefullness_2" : "categorization_rating",
    "feature_usefullness_3" : "category_sidebar_rating"
}

### Rename the columns

In [84]:
df = df.rename(rename_columns, axis=1)

#### Visualize the changes

In [85]:
a = df[["ResponseId","t3_1","t3_2","t3_3","t3_4"]]
a.dropna()

,ResponseId,t3_1,t3_2,t3_3,t3_4
0,Response ID,Based on the website variation you have just t...,Based on the website variation you have just t...,Based on the website variation you have just t...,Based on the website variation you have just t...
1,"{""ImportId"":""_recordId""}","{""ImportId"":""QID95_1""}","{""ImportId"":""QID95_2""}","{""ImportId"":""QID95_3""}","{""ImportId"":""QID95_4""}"
3,R_28J2ielzrCfvy71,0,1,2,3
5,R_1lsl8kCjFiBTyzK,3,1,1,1
9,R_UfKVwc8Ysy6pN1n,2,1,1,1
10,R_1cTAX495KgNw4zW,1,2,2,1
11,R_3hDv6lQBiaJSbmx,2,2,-1,3
13,R_3JaYb1pOqZxJWjR,0,-1,-1,0
14,R_4GvYKPN7O7VmLE5,2,2,2,2
15,R_BVzB1em5zins9Ed,1,1,1,1


### Drop 1st and 2nd rows containing redundant data(question text, import id)

In [86]:
df = df.drop(df.index[0:2])

### Combine list of columns to change the data type

In [87]:
correct_dtypes = {
    "t3_1": float,
    "t3_2": float,
    "t3_3": float,
    "t3_4": float,
    "t6_1": float,
    "t6_2": float,
    "t6_3": float,
    "t6_4": float,
    "t9_1": float,
    "t9_2": float,
    "t9_3": float,
    "t9_4": float,
    "t12_1":float,
    "t12_2":float,
    "t12_3":float,
    "t12_4":float,
    "t15_1":float,
    "t15_2":float,
    "t15_3":float,
    "t15_4":float,
    "t18_1":float,
    "t18_2":float,
    "t18_3":float,
    "t18_4":float,
    "t21_1":float,
    "t21_2":float,
    "t21_3":float,
    "t21_4":float,
    "t24_1":float,
    "t24_2":float,
    "t24_3":float,
    "t24_4":float,
    "t27_1":float,
    "t27_2":float,
    "t27_3":float,
    "t27_4":float,
    "t30_1":float,
    "t30_2":float,
    "t30_3":float,
    "t30_4":float,
    "t33_1":float,
    "t33_2":float,
    "t33_3":float,
    "t33_4":float,
    "t36_1":float,
    "t36_2":float,
    "t36_3":float,
    "t36_4":float,
    "sidebar_rating":float,
    "categorization_rating":float,
    "category_sidebar_rating":float
}

### Change the data type

In [88]:
df = df.astype(correct_dtypes)

### Create functions to combine the 4 questions for ease_of_use using System Usability Scale method

In [89]:
def process_t3(row):
    if row.IV1 == "fixed-sidebar":
        #return 2.5*( (row.t3_1+4-1) + (7 - row.t3_2 - 4) + (row.t3_3 + 4-1) + (7 - row.t3_4 - 4) )
        return (row.t3_1, row.t3_2, row.t3_3, row.t3_4)
    else:
        return None
    
def process_t6(row):
    if row.IV1 == "fixed-sidebar":
        #return 2.5*( (row.t6_1 + 4 - 1) + (7 - row.t6_2 - 4) + (row.t6_3 + 4 - 1) + (7 - row.t6_4 - 4) )
        return (row.t6_1, row.t6_2, row.t6_3, row.t6_4)
    else:
        return None

def process_t9(row):
    if row.IV1 == "fixed-sidebar":
        #return 2.5*( (row.t9_1+4-1) + (7 - row.t9_2 - 4) + (row.t9_3 + 4 -1) + (7 - row.t9_4 - 4) )
        return (row.t9_1, row.t9_2, row.t9_3, row.t9_4)
    else:
        return None
        
def process_t12(row):
    print(row)
    if row.IV1 == "fixed-sidebar":
        #return 2.5*( (row.t12_1 + 4 -1) + (7 - row.t12_2 - 4) + (row.t12_3 + 4 -1) + (7 - row.t12_4 - 4) )
        return (row.t12_1, row.t12_2, row.t12_3, row.t12_4)
    else:
        return None
    
def process_t15(row):
    if row.IV1 == "fixed-sidebar":
        #return 2.5*( (row.t15_1 + 4 -1) + (7 - row.t15_2 - 4) + (row.t15_3 + 4 -1) + (7 - row.t15_4 - 4) )
        return (row.t15_1, row.t15_2, row.t15_3, row.t15_4)
    else:
        return None
    
def process_t18(row):
    print(row)
    if row.IV1 == "fixed-sidebar":
        #return 2.5*( (row.t18_1 + 4 -1) + (7 - row.t18_2 - 4) + (row.t18_3 + 4 -1) + (7 - row.t18_4 - 4) )
        return (row.t18_1, row.t18_2, row.t18_3, row.t18_4)
    else:
        return None

def process_t21(row):
    if row.IV1 == "floating-sidebar":
        return (row.t21_1, row.t21_2, row.t21_3, row.t21_4)
        #return 2.5*( (row.t21_1 + 4 -1) + (7 - row.t21_2 - 4) + (row.t21_3 + 4 -1) + (7 - row.t21_4 - 4) )
    else:
        return None
    
def process_t24(row):
    if row.IV1 == "floating-sidebar":
        return (row.t24_1, row.t24_2, row.t24_3, row.t24_4)
        #return 2.5*( (row.t24_1 + 4 -1) + (7 - row.t24_2 - 4) + (row.t24_3 + 4 -1) + (7 - row.t24_4 - 4) )
    else:
        return None

def process_t27(row):
    if row.IV1 == "floating-sidebar":
        return (row.t27_1, row.t27_2, row.t27_3, row.t27_4)
        #return 2.5*( (row.t27_1 + 4 -1) + (7 - row.t27_2 - 4) + (row.t27_3 + 4 -1) + (7 - row.t27_4 - 4) )
    else:
        return None
        
def process_t30(row):
    if row.IV1 == "floating-sidebar":
        return (row.t30_1, row.t30_2, row.t30_3, row.t30_4)
        #return 2.5*( (row.t30_1 + 4 -1) + (7 - row.t30_2 - 4) + (row.t30_3 + 4 -1) + (7 - row.t30_4 - 4) )
    else:
        return None
    
def process_t33(row):
    if row.IV1 == "floating-sidebar":
        return (row.t33_1, row.t33_2, row.t33_3, row.t33_4)
        #return 2.5*( (row.t33_1 + 4 -1) + (7 - row.t33_2 - 4) + (row.t33_3 + 4 -1) + (7 - row.t33_4 - 4) )
    else:
        return None
    
def process_t36(row):
    if row.IV1 == "floating-sidebar":
        return (row.t36_1, row.t36_2, row.t36_3, row.t36_4)
        #return 2.5*( (row.t36_1 + 4 -1) + (7 - row.t36_2 - 4) + (row.t36_3 + 4 -1) + (7 - row.t36_4 - 4) )
    else:
        return None

### Apply the previous methods to trial question and combine them together into 1 metric
### Questions were asked after every 3 trials, so broadcast the same answers for previous 2 trials

In [90]:
df["t3"] = df.apply(lambda row: process_t3(row), axis=1)
df["t1"] = df.apply(lambda row:row.t3, axis=1)
df["t2"] = df.apply(lambda row:row.t3, axis=1)

df["t6"] = df.apply(lambda row: process_t6(row), axis=1)
df["t4"] = df.apply(lambda row:row.t6, axis=1)
df["t5"] = df.apply(lambda row:row.t6, axis=1)

df["t9"] = df.apply(lambda row: process_t9(row), axis=1)
df["t7"] = df.apply(lambda row:row.t9, axis=1)
df["t8"] = df.apply(lambda row:row.t9, axis=1)

df["t12"] = df.apply(lambda row: process_t12(row), axis=1)
df["t10"] = df.apply(lambda row:row.t12, axis=1)
df["t11"] = df.apply(lambda row:row.t12, axis=1)

df["t15"] = df.apply(lambda row: process_t15(row), axis=1)
df["t13"] = df.apply(lambda row:row.t15, axis=1)
df["t14"] = df.apply(lambda row:row.t15, axis=1)

df["t18"] = df.apply(lambda row: process_t18(row), axis=1)
df["t16"] = df.apply(lambda row:row.t18, axis=1)
df["t17"] = df.apply(lambda row:row.t18, axis=1)

df["t21"] = df.apply(lambda row: process_t21(row), axis=1)
df["t19"] = df.apply(lambda row:row.t21, axis=1)
df["t20"] = df.apply(lambda row:row.t21, axis=1)

df["t24"] = df.apply(lambda row: process_t24(row), axis=1)
df["t22"] = df.apply(lambda row:row.t24, axis=1)
df["t23"] = df.apply(lambda row:row.t24, axis=1)

df["t27"] = df.apply(lambda row: process_t27(row), axis=1)
df["t25"] = df.apply(lambda row:row.t27, axis=1)
df["t26"] = df.apply(lambda row:row.t27, axis=1)

df["t30"] = df.apply(lambda row: process_t30(row), axis=1)
df["t28"] = df.apply(lambda row:row.t30, axis=1)
df["t29"] = df.apply(lambda row:row.t30, axis=1)

df["t33"] = df.apply(lambda row: process_t33(row), axis=1)
df["t31"] = df.apply(lambda row:row.t33, axis=1)
df["t32"] = df.apply(lambda row:row.t33, axis=1)

df["t36"] = df.apply(lambda row: process_t36(row), axis=1)
df["t34"] = df.apply(lambda row:row.t36, axis=1)
df["t35"] = df.apply(lambda row:row.t36, axis=1)

Progress                               100
Duration (in seconds)                  565
Finished                                 1
ResponseId               R_1jg3libD6KGk9H6
pri_exp                                  1
                               ...        
t4                                    None
t5                                    None
t9                                    None
t7                                    None
t8                                    None
Name: 2, Length: 78, dtype: object
Progress                                  100
Duration (in seconds)                     100
Finished                                    1
ResponseId                  R_28J2ielzrCfvy71
pri_exp                                     1
                                 ...         
t4                       (1.0, 2.0, 2.0, 2.0)
t5                       (1.0, 2.0, 2.0, 2.0)
t9                       (3.0, 2.0, 2.0, 1.0)
t7                       (3.0, 2.0, 2.0, 1.0)
t8                       (3.0, 2

#### Visualize the changes

In [91]:
a = df[["ResponseId","t3_1","t3_2","t3_3","t3_4","t3"]]
a.dropna()

,ResponseId,t3_1,t3_2,t3_3,t3_4,t3
3,R_28J2ielzrCfvy71,0.0,1.0,2.0,3.0,"(0.0, 1.0, 2.0, 3.0)"
5,R_1lsl8kCjFiBTyzK,3.0,1.0,1.0,1.0,"(3.0, 1.0, 1.0, 1.0)"
9,R_UfKVwc8Ysy6pN1n,2.0,1.0,1.0,1.0,"(2.0, 1.0, 1.0, 1.0)"
10,R_1cTAX495KgNw4zW,1.0,2.0,2.0,1.0,"(1.0, 2.0, 2.0, 1.0)"
11,R_3hDv6lQBiaJSbmx,2.0,2.0,-1.0,3.0,"(2.0, 2.0, -1.0, 3.0)"
13,R_3JaYb1pOqZxJWjR,0.0,-1.0,-1.0,0.0,"(0.0, -1.0, -1.0, 0.0)"
14,R_4GvYKPN7O7VmLE5,2.0,2.0,2.0,2.0,"(2.0, 2.0, 2.0, 2.0)"
15,R_BVzB1em5zins9Ed,1.0,1.0,1.0,1.0,"(1.0, 1.0, 1.0, 1.0)"
16,R_3JEPAzSSS7vl7m8,1.0,1.0,1.0,1.0,"(1.0, 1.0, 1.0, 1.0)"
20,R_R9rAxbkMUkt1JD3,-3.0,-3.0,-3.0,-3.0,"(-3.0, -3.0, -3.0, -3.0)"


### Drop the individual columns for each question since their data has been combined into 1 single column

In [92]:
drop_t_columns = {
"t3_1",
"t3_2",
"t3_3",
"t3_4",
"t6_1",
"t6_2",
"t6_3",
"t6_4",
"t9_1",
"t9_2",
"t9_3",
"t9_4",
"t12_1",
"t12_2",
"t12_3",
"t12_4",
"t15_1",
"t15_2",
"t15_3",
"t15_4",
"t18_1",
"t18_2",
"t18_3",
"t18_4",
"t21_1",
"t21_2",
"t21_3",
"t21_4",
"t24_1",
"t24_2",
"t24_3",
"t24_4",
"t27_1",
"t27_2",
"t27_3",
"t27_4",
"t30_1",
"t30_2",
"t30_3",
"t30_4",
"t33_1",
"t33_2",
"t33_3",
"t33_4",
"t36_1",
"t36_2",
"t36_3",
"t36_4"
}

In [93]:
df = df.drop(drop_t_columns, axis=1)

In [94]:
a = df[["ResponseId","t3","t6","t9","t12","t15"]]
a.dropna()

,ResponseId,t3,t6,t9,t12,t15
3,R_28J2ielzrCfvy71,"(0.0, 1.0, 2.0, 3.0)","(1.0, 2.0, 2.0, 2.0)","(3.0, 2.0, 2.0, 1.0)","(2.0, 2.0, 1.0, 0.0)","(3.0, 2.0, 1.0, 1.0)"
5,R_1lsl8kCjFiBTyzK,"(3.0, 1.0, 1.0, 1.0)","(3.0, 3.0, 3.0, 3.0)","(3.0, -3.0, -3.0, 3.0)","(3.0, 3.0, 3.0, 3.0)","(3.0, 3.0, 3.0, 3.0)"
9,R_UfKVwc8Ysy6pN1n,"(2.0, 1.0, 1.0, 1.0)","(2.0, 2.0, 2.0, 2.0)","(1.0, 2.0, 0.0, 1.0)","(2.0, 2.0, 2.0, 2.0)","(2.0, 2.0, 2.0, 2.0)"
10,R_1cTAX495KgNw4zW,"(1.0, 2.0, 2.0, 1.0)","(1.0, 1.0, 2.0, 1.0)","(1.0, 1.0, 0.0, 1.0)","(1.0, 1.0, 2.0, 1.0)","(2.0, 1.0, 2.0, 2.0)"
11,R_3hDv6lQBiaJSbmx,"(2.0, 2.0, -1.0, 3.0)","(1.0, 2.0, -2.0, 1.0)","(2.0, 2.0, -1.0, 2.0)","(2.0, 3.0, 2.0, 3.0)","(2.0, 2.0, 2.0, 3.0)"
13,R_3JaYb1pOqZxJWjR,"(0.0, -1.0, -1.0, 0.0)","(0.0, 0.0, -1.0, 0.0)","(-2.0, -2.0, -2.0, -2.0)","(0.0, 0.0, 0.0, 0.0)","(1.0, 1.0, 1.0, 1.0)"
14,R_4GvYKPN7O7VmLE5,"(2.0, 2.0, 2.0, 2.0)","(2.0, 2.0, 2.0, 2.0)","(1.0, -1.0, -1.0, 2.0)","(2.0, 2.0, 2.0, 2.0)","(2.0, 3.0, 3.0, 3.0)"
15,R_BVzB1em5zins9Ed,"(1.0, 1.0, 1.0, 1.0)","(3.0, 3.0, 3.0, 3.0)","(0.0, 0.0, -1.0, 0.0)","(2.0, 2.0, 2.0, 2.0)","(3.0, 3.0, 3.0, 3.0)"
16,R_3JEPAzSSS7vl7m8,"(1.0, 1.0, 1.0, 1.0)","(1.0, -2.0, 1.0, 2.0)","(1.0, -2.0, -2.0, -1.0)","(2.0, 2.0, 2.0, 2.0)","(1.0, 1.0, 1.0, 2.0)"
20,R_R9rAxbkMUkt1JD3,"(-3.0, -3.0, -3.0, -3.0)","(-3.0, -3.0, -3.0, -3.0)","(-3.0, -3.0, -3.0, -3.0)","(-2.0, -2.0, -3.0, -2.0)","(-1.0, -1.0, 0.0, -1.0)"


### Drop some more unused columns

In [95]:
df = df.drop(["platform","Duration (in seconds)"], axis=1)

### Convert from wide to long form

In [96]:
df2 = pd.melt(df, id_vars=["Progress", "Finished","ResponseId","pri_exp","buy_check","browser_check","task_check","age_group","gender","computer_use_confide","sweet_food_affinity","computer_os","sidebar_rating","categorization_rating","category_sidebar_rating","IV1","arrangement","sequence" ,"ecomm_frequence"  ], value_vars = ["t1","t2","t3","t4","t5","t6","t7","t8","t9","t10","t11","t12","t13","t14","t15","t16","t17","t18","t19","t21","t22","t23","t24","t25","t26","t27","t28","t29","t30","t31","t32","t33","t34","t35","t36"], var_name="trial_id",value_name='ease_of_use')

### Remove the rows with NA in ease_of_use column

In [97]:
df3 = df2[df2['ease_of_use'].notna()]

In [98]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 423 entries, 1 to 1012
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Progress                 423 non-null    object 
 1   Finished                 423 non-null    object 
 2   ResponseId               423 non-null    object 
 3   pri_exp                  423 non-null    object 
 4   buy_check                423 non-null    object 
 5   browser_check            423 non-null    object 
 6   task_check               423 non-null    object 
 7   age_group                423 non-null    object 
 8   gender                   423 non-null    object 
 9   computer_use_confide     423 non-null    object 
 10  sweet_food_affinity      423 non-null    object 
 11  computer_os              423 non-null    object 
 12  sidebar_rating           423 non-null    float64
 13  categorization_rating    423 non-null    float64
 14  category_sidebar_rating  

### Remove some more unused columns

In [99]:
df3 = df3.drop(["Progress","Finished","task_check","browser_check","sequence"], axis=1)

### Convert data type of trial_id columns to int

In [100]:
df3["trial_id"] = df3["trial_id"].str.replace(r'\D+', '').astype('int')

<ipython-input-100-7e6a4762c9ee>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df3["trial_id"] = df3["trial_id"].str.replace(r'\D+', '').astype('int')


### Function for getting the level of IV2 and IV3 using the trial_id

In [101]:
def make_iv2(row):
    if any(lower<=row.trial_id<=upper for (lower, upper) in [(1,3), (10,12),(19,21),(28,30)]):
        return "category_sidebar"
    elif any(lower<=row.trial_id<=upper for (lower, upper) in [(8,9), (16,18),(25,27),(34,26)]):
        return "original"
    else:
        return "category_sidebar_with_sub-categorization"

def make_iv3(row):
    if any(lower<=row.trial_id<=upper for (lower, upper) in [(1,9), (19,27)]):
        return "6-cupcakes"
    else:
        return "3-cupcakes"        

### Apply the functions previously defined to create columns for IV2 and IV3

In [102]:
df3["IV2"] = df3.apply(lambda row:make_iv2(row),axis=1)
df3["IV3"] = df3.apply(lambda row:make_iv3(row),axis=1)

In [103]:
#print(df3[["ResponseId","IV1","IV2","IV3","trial_id","ease_of_use"]].to_string())

In [104]:
a = df3[["ResponseId","IV1","IV2","IV3","trial_id","ease_of_use"]]
a.head(10)

,ResponseId,IV1,IV2,IV3,trial_id,ease_of_use
1,R_28J2ielzrCfvy71,fixed-sidebar,category_sidebar,6-cupcakes,1,"(0.0, 1.0, 2.0, 3.0)"
3,R_1lsl8kCjFiBTyzK,fixed-sidebar,category_sidebar,6-cupcakes,1,"(3.0, 1.0, 1.0, 1.0)"
7,R_UfKVwc8Ysy6pN1n,fixed-sidebar,category_sidebar,6-cupcakes,1,"(2.0, 1.0, 1.0, 1.0)"
8,R_1cTAX495KgNw4zW,fixed-sidebar,category_sidebar,6-cupcakes,1,"(1.0, 2.0, 2.0, 1.0)"
9,R_3hDv6lQBiaJSbmx,fixed-sidebar,category_sidebar,6-cupcakes,1,"(2.0, 2.0, -1.0, 3.0)"
11,R_3JaYb1pOqZxJWjR,fixed-sidebar,category_sidebar,6-cupcakes,1,"(0.0, -1.0, -1.0, 0.0)"
12,R_4GvYKPN7O7VmLE5,fixed-sidebar,category_sidebar,6-cupcakes,1,"(2.0, 2.0, 2.0, 2.0)"
13,R_BVzB1em5zins9Ed,fixed-sidebar,category_sidebar,6-cupcakes,1,"(1.0, 1.0, 1.0, 1.0)"
14,R_3JEPAzSSS7vl7m8,fixed-sidebar,category_sidebar,6-cupcakes,1,"(1.0, 1.0, 1.0, 1.0)"
18,R_R9rAxbkMUkt1JD3,fixed-sidebar,category_sidebar,6-cupcakes,1,"(-3.0, -3.0, -3.0, -3.0)"


### Drop some more unused columns

In [105]:
df3 = df3.drop(["pri_exp","buy_check"], axis=1)

### List of final columns

In [106]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 423 entries, 1 to 1012
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ResponseId               423 non-null    object 
 1   age_group                423 non-null    object 
 2   gender                   423 non-null    object 
 3   computer_use_confide     423 non-null    object 
 4   sweet_food_affinity      423 non-null    object 
 5   computer_os              423 non-null    object 
 6   sidebar_rating           423 non-null    float64
 7   categorization_rating    423 non-null    float64
 8   category_sidebar_rating  423 non-null    float64
 9   IV1                      423 non-null    object 
 10  arrangement              423 non-null    object 
 11  ecomm_frequence          423 non-null    object 
 12  trial_id                 423 non-null    int32  
 13  ease_of_use              423 non-null    object 
 14  IV2                      

### First 5 rows in cleaned data

In [107]:
df3.head()

,ResponseId,age_group,gender,computer_use_confide,sweet_food_affinity,computer_os,sidebar_rating,categorization_rating,category_sidebar_rating,IV1,arrangement,ecomm_frequence,trial_id,ease_of_use,IV2,IV3
1,R_28J2ielzrCfvy71,2,1,3,-1,1,7.0,6.0,5.0,fixed-sidebar,3,2,1,"(0.0, 1.0, 2.0, 3.0)",category_sidebar,6-cupcakes
3,R_1lsl8kCjFiBTyzK,2,1,3,3,1,7.0,7.0,7.0,fixed-sidebar,6,2,1,"(3.0, 1.0, 1.0, 1.0)",category_sidebar,6-cupcakes
7,R_UfKVwc8Ysy6pN1n,2,2,2,2,1,7.0,3.0,2.0,fixed-sidebar,3,2,1,"(2.0, 1.0, 1.0, 1.0)",category_sidebar,6-cupcakes
8,R_1cTAX495KgNw4zW,2,1,2,-1,1,6.0,5.0,7.0,fixed-sidebar,6,3,1,"(1.0, 2.0, 2.0, 1.0)",category_sidebar,6-cupcakes
9,R_3hDv6lQBiaJSbmx,2,2,3,1,1,6.0,5.0,5.0,fixed-sidebar,2,1,1,"(2.0, 2.0, -1.0, 3.0)",category_sidebar,6-cupcakes


In [108]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 423 entries, 1 to 1012
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ResponseId               423 non-null    object 
 1   age_group                423 non-null    object 
 2   gender                   423 non-null    object 
 3   computer_use_confide     423 non-null    object 
 4   sweet_food_affinity      423 non-null    object 
 5   computer_os              423 non-null    object 
 6   sidebar_rating           423 non-null    float64
 7   categorization_rating    423 non-null    float64
 8   category_sidebar_rating  423 non-null    float64
 9   IV1                      423 non-null    object 
 10  arrangement              423 non-null    object 
 11  ecomm_frequence          423 non-null    object 
 12  trial_id                 423 non-null    int32  
 13  ease_of_use              423 non-null    object 
 14  IV2                      

### Recode values in age group

In [109]:
df3.age_group.unique()

array(['2'], dtype=object)

In [110]:
df3["age_group"] = df3["age_group"].replace(["1","2","3","4"], ["less than 20", "between 20 and 30", "between 30 and 40","more than 40"])

In [111]:
df3.head()

,ResponseId,age_group,gender,computer_use_confide,sweet_food_affinity,computer_os,sidebar_rating,categorization_rating,category_sidebar_rating,IV1,arrangement,ecomm_frequence,trial_id,ease_of_use,IV2,IV3
1,R_28J2ielzrCfvy71,between 20 and 30,1,3,-1,1,7.0,6.0,5.0,fixed-sidebar,3,2,1,"(0.0, 1.0, 2.0, 3.0)",category_sidebar,6-cupcakes
3,R_1lsl8kCjFiBTyzK,between 20 and 30,1,3,3,1,7.0,7.0,7.0,fixed-sidebar,6,2,1,"(3.0, 1.0, 1.0, 1.0)",category_sidebar,6-cupcakes
7,R_UfKVwc8Ysy6pN1n,between 20 and 30,2,2,2,1,7.0,3.0,2.0,fixed-sidebar,3,2,1,"(2.0, 1.0, 1.0, 1.0)",category_sidebar,6-cupcakes
8,R_1cTAX495KgNw4zW,between 20 and 30,1,2,-1,1,6.0,5.0,7.0,fixed-sidebar,6,3,1,"(1.0, 2.0, 2.0, 1.0)",category_sidebar,6-cupcakes
9,R_3hDv6lQBiaJSbmx,between 20 and 30,2,3,1,1,6.0,5.0,5.0,fixed-sidebar,2,1,1,"(2.0, 2.0, -1.0, 3.0)",category_sidebar,6-cupcakes


### Recode gender

In [112]:
df3["gender"] = df3["gender"].replace(["1","2","3","4"], ["Male", "Female", "Non-binary","Unspecified"])

In [113]:
df3.head()

,ResponseId,age_group,gender,computer_use_confide,sweet_food_affinity,computer_os,sidebar_rating,categorization_rating,category_sidebar_rating,IV1,arrangement,ecomm_frequence,trial_id,ease_of_use,IV2,IV3
1,R_28J2ielzrCfvy71,between 20 and 30,Male,3,-1,1,7.0,6.0,5.0,fixed-sidebar,3,2,1,"(0.0, 1.0, 2.0, 3.0)",category_sidebar,6-cupcakes
3,R_1lsl8kCjFiBTyzK,between 20 and 30,Male,3,3,1,7.0,7.0,7.0,fixed-sidebar,6,2,1,"(3.0, 1.0, 1.0, 1.0)",category_sidebar,6-cupcakes
7,R_UfKVwc8Ysy6pN1n,between 20 and 30,Female,2,2,1,7.0,3.0,2.0,fixed-sidebar,3,2,1,"(2.0, 1.0, 1.0, 1.0)",category_sidebar,6-cupcakes
8,R_1cTAX495KgNw4zW,between 20 and 30,Male,2,-1,1,6.0,5.0,7.0,fixed-sidebar,6,3,1,"(1.0, 2.0, 2.0, 1.0)",category_sidebar,6-cupcakes
9,R_3hDv6lQBiaJSbmx,between 20 and 30,Female,3,1,1,6.0,5.0,5.0,fixed-sidebar,2,1,1,"(2.0, 2.0, -1.0, 3.0)",category_sidebar,6-cupcakes


### Recode computer OS

In [114]:
df3["computer_os"] = df3["computer_os"].replace(["1","2","4"], ["macOs", "Windows", "Other"])

In [115]:
df3.head()

,ResponseId,age_group,gender,computer_use_confide,sweet_food_affinity,computer_os,sidebar_rating,categorization_rating,category_sidebar_rating,IV1,arrangement,ecomm_frequence,trial_id,ease_of_use,IV2,IV3
1,R_28J2ielzrCfvy71,between 20 and 30,Male,3,-1,macOs,7.0,6.0,5.0,fixed-sidebar,3,2,1,"(0.0, 1.0, 2.0, 3.0)",category_sidebar,6-cupcakes
3,R_1lsl8kCjFiBTyzK,between 20 and 30,Male,3,3,macOs,7.0,7.0,7.0,fixed-sidebar,6,2,1,"(3.0, 1.0, 1.0, 1.0)",category_sidebar,6-cupcakes
7,R_UfKVwc8Ysy6pN1n,between 20 and 30,Female,2,2,macOs,7.0,3.0,2.0,fixed-sidebar,3,2,1,"(2.0, 1.0, 1.0, 1.0)",category_sidebar,6-cupcakes
8,R_1cTAX495KgNw4zW,between 20 and 30,Male,2,-1,macOs,6.0,5.0,7.0,fixed-sidebar,6,3,1,"(1.0, 2.0, 2.0, 1.0)",category_sidebar,6-cupcakes
9,R_3hDv6lQBiaJSbmx,between 20 and 30,Female,3,1,macOs,6.0,5.0,5.0,fixed-sidebar,2,1,1,"(2.0, 2.0, -1.0, 3.0)",category_sidebar,6-cupcakes


### Recode computer use confidence

In [116]:
df3["computer_use_confide"] = df3["computer_use_confide"].replace(["-3","-2","-1","0","1","2","3"], ["Not at all confident", "Not confident", "Somewhat not confident","Neither confident nor uncofident","Somewhat confident","Confident","Very confident"])

In [117]:
df3.head()

,ResponseId,age_group,gender,computer_use_confide,sweet_food_affinity,computer_os,sidebar_rating,categorization_rating,category_sidebar_rating,IV1,arrangement,ecomm_frequence,trial_id,ease_of_use,IV2,IV3
1,R_28J2ielzrCfvy71,between 20 and 30,Male,Very confident,-1,macOs,7.0,6.0,5.0,fixed-sidebar,3,2,1,"(0.0, 1.0, 2.0, 3.0)",category_sidebar,6-cupcakes
3,R_1lsl8kCjFiBTyzK,between 20 and 30,Male,Very confident,3,macOs,7.0,7.0,7.0,fixed-sidebar,6,2,1,"(3.0, 1.0, 1.0, 1.0)",category_sidebar,6-cupcakes
7,R_UfKVwc8Ysy6pN1n,between 20 and 30,Female,Confident,2,macOs,7.0,3.0,2.0,fixed-sidebar,3,2,1,"(2.0, 1.0, 1.0, 1.0)",category_sidebar,6-cupcakes
8,R_1cTAX495KgNw4zW,between 20 and 30,Male,Confident,-1,macOs,6.0,5.0,7.0,fixed-sidebar,6,3,1,"(1.0, 2.0, 2.0, 1.0)",category_sidebar,6-cupcakes
9,R_3hDv6lQBiaJSbmx,between 20 and 30,Female,Very confident,1,macOs,6.0,5.0,5.0,fixed-sidebar,2,1,1,"(2.0, 2.0, -1.0, 3.0)",category_sidebar,6-cupcakes


### Recode affinity to sweet foods

In [118]:
df3["sweet_food_affinity"] = df3["sweet_food_affinity"].replace(["-3","-2","-1","0","1","2","3"], ["Strongly dislike", "Dislike", "Somewhat dislike","Neither like nor dislike","Somewhat like","Like","Like a lot"])

In [119]:
df3.head()

,ResponseId,age_group,gender,computer_use_confide,sweet_food_affinity,computer_os,sidebar_rating,categorization_rating,category_sidebar_rating,IV1,arrangement,ecomm_frequence,trial_id,ease_of_use,IV2,IV3
1,R_28J2ielzrCfvy71,between 20 and 30,Male,Very confident,Somewhat dislike,macOs,7.0,6.0,5.0,fixed-sidebar,3,2,1,"(0.0, 1.0, 2.0, 3.0)",category_sidebar,6-cupcakes
3,R_1lsl8kCjFiBTyzK,between 20 and 30,Male,Very confident,Like a lot,macOs,7.0,7.0,7.0,fixed-sidebar,6,2,1,"(3.0, 1.0, 1.0, 1.0)",category_sidebar,6-cupcakes
7,R_UfKVwc8Ysy6pN1n,between 20 and 30,Female,Confident,Like,macOs,7.0,3.0,2.0,fixed-sidebar,3,2,1,"(2.0, 1.0, 1.0, 1.0)",category_sidebar,6-cupcakes
8,R_1cTAX495KgNw4zW,between 20 and 30,Male,Confident,Somewhat dislike,macOs,6.0,5.0,7.0,fixed-sidebar,6,3,1,"(1.0, 2.0, 2.0, 1.0)",category_sidebar,6-cupcakes
9,R_3hDv6lQBiaJSbmx,between 20 and 30,Female,Very confident,Somewhat like,macOs,6.0,5.0,5.0,fixed-sidebar,2,1,1,"(2.0, 2.0, -1.0, 3.0)",category_sidebar,6-cupcakes


### Recode ecommerce use frequency

In [120]:
df3["ecomm_frequence"] = df3["ecomm_frequence"].replace(["1","2","3"], ["Last purchased within 1 week", "Last purchased between 1 week and 1 month", "Last purchansed more than 1 month ago"])

In [121]:
df3.head()

,ResponseId,age_group,gender,computer_use_confide,sweet_food_affinity,computer_os,sidebar_rating,categorization_rating,category_sidebar_rating,IV1,arrangement,ecomm_frequence,trial_id,ease_of_use,IV2,IV3
1,R_28J2ielzrCfvy71,between 20 and 30,Male,Very confident,Somewhat dislike,macOs,7.0,6.0,5.0,fixed-sidebar,3,Last purchased between 1 week and 1 month,1,"(0.0, 1.0, 2.0, 3.0)",category_sidebar,6-cupcakes
3,R_1lsl8kCjFiBTyzK,between 20 and 30,Male,Very confident,Like a lot,macOs,7.0,7.0,7.0,fixed-sidebar,6,Last purchased between 1 week and 1 month,1,"(3.0, 1.0, 1.0, 1.0)",category_sidebar,6-cupcakes
7,R_UfKVwc8Ysy6pN1n,between 20 and 30,Female,Confident,Like,macOs,7.0,3.0,2.0,fixed-sidebar,3,Last purchased between 1 week and 1 month,1,"(2.0, 1.0, 1.0, 1.0)",category_sidebar,6-cupcakes
8,R_1cTAX495KgNw4zW,between 20 and 30,Male,Confident,Somewhat dislike,macOs,6.0,5.0,7.0,fixed-sidebar,6,Last purchansed more than 1 month ago,1,"(1.0, 2.0, 2.0, 1.0)",category_sidebar,6-cupcakes
9,R_3hDv6lQBiaJSbmx,between 20 and 30,Female,Very confident,Somewhat like,macOs,6.0,5.0,5.0,fixed-sidebar,2,Last purchased within 1 week,1,"(2.0, 2.0, -1.0, 3.0)",category_sidebar,6-cupcakes


### Split ease of use values

In [122]:
def process_ease_of_use(row, index):
    return row.ease_of_use[index]

In [123]:
df3["ease_of_use_task_completion"] = df3.apply(lambda row: process_ease_of_use(row,0), axis=1)
df3["ease_of_use_navigate_to_browsing"] = df3.apply(lambda row: process_ease_of_use(row,1), axis=1)
df3["ease_of_use_find_cupcake"] = df3.apply(lambda row: process_ease_of_use(row,2), axis=1)
df3["ease_of_use_total_time"] = df3.apply(lambda row: process_ease_of_use(row,3), axis=1)

In [124]:
a = df3[["ResponseId","ease_of_use","ease_of_use_task_completion","ease_of_use_navigate_to_browsing","ease_of_use_find_cupcake","ease_of_use_total_time"]]
a.head(10)

,ResponseId,ease_of_use,ease_of_use_task_completion,ease_of_use_navigate_to_browsing,ease_of_use_find_cupcake,ease_of_use_total_time
1,R_28J2ielzrCfvy71,"(0.0, 1.0, 2.0, 3.0)",0.0,1.0,2.0,3.0
3,R_1lsl8kCjFiBTyzK,"(3.0, 1.0, 1.0, 1.0)",3.0,1.0,1.0,1.0
7,R_UfKVwc8Ysy6pN1n,"(2.0, 1.0, 1.0, 1.0)",2.0,1.0,1.0,1.0
8,R_1cTAX495KgNw4zW,"(1.0, 2.0, 2.0, 1.0)",1.0,2.0,2.0,1.0
9,R_3hDv6lQBiaJSbmx,"(2.0, 2.0, -1.0, 3.0)",2.0,2.0,-1.0,3.0
11,R_3JaYb1pOqZxJWjR,"(0.0, -1.0, -1.0, 0.0)",0.0,-1.0,-1.0,0.0
12,R_4GvYKPN7O7VmLE5,"(2.0, 2.0, 2.0, 2.0)",2.0,2.0,2.0,2.0
13,R_BVzB1em5zins9Ed,"(1.0, 1.0, 1.0, 1.0)",1.0,1.0,1.0,1.0
14,R_3JEPAzSSS7vl7m8,"(1.0, 1.0, 1.0, 1.0)",1.0,1.0,1.0,1.0
18,R_R9rAxbkMUkt1JD3,"(-3.0, -3.0, -3.0, -3.0)",-3.0,-3.0,-3.0,-3.0


In [125]:
df3 = df3.drop(["ease_of_use"], axis=1)

In [126]:
df3.head()

,ResponseId,age_group,gender,computer_use_confide,sweet_food_affinity,computer_os,sidebar_rating,categorization_rating,category_sidebar_rating,IV1,arrangement,ecomm_frequence,trial_id,IV2,IV3,ease_of_use_task_completion,ease_of_use_navigate_to_browsing,ease_of_use_find_cupcake,ease_of_use_total_time
1,R_28J2ielzrCfvy71,between 20 and 30,Male,Very confident,Somewhat dislike,macOs,7.0,6.0,5.0,fixed-sidebar,3,Last purchased between 1 week and 1 month,1,category_sidebar,6-cupcakes,0.0,1.0,2.0,3.0
3,R_1lsl8kCjFiBTyzK,between 20 and 30,Male,Very confident,Like a lot,macOs,7.0,7.0,7.0,fixed-sidebar,6,Last purchased between 1 week and 1 month,1,category_sidebar,6-cupcakes,3.0,1.0,1.0,1.0
7,R_UfKVwc8Ysy6pN1n,between 20 and 30,Female,Confident,Like,macOs,7.0,3.0,2.0,fixed-sidebar,3,Last purchased between 1 week and 1 month,1,category_sidebar,6-cupcakes,2.0,1.0,1.0,1.0
8,R_1cTAX495KgNw4zW,between 20 and 30,Male,Confident,Somewhat dislike,macOs,6.0,5.0,7.0,fixed-sidebar,6,Last purchansed more than 1 month ago,1,category_sidebar,6-cupcakes,1.0,2.0,2.0,1.0
9,R_3hDv6lQBiaJSbmx,between 20 and 30,Female,Very confident,Somewhat like,macOs,6.0,5.0,5.0,fixed-sidebar,2,Last purchased within 1 week,1,category_sidebar,6-cupcakes,2.0,2.0,-1.0,3.0


In [128]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 423 entries, 1 to 1012
Data columns (total 19 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ResponseId                        423 non-null    object 
 1   age_group                         423 non-null    object 
 2   gender                            423 non-null    object 
 3   computer_use_confide              423 non-null    object 
 4   sweet_food_affinity               423 non-null    object 
 5   computer_os                       423 non-null    object 
 6   sidebar_rating                    423 non-null    float64
 7   categorization_rating             423 non-null    float64
 8   category_sidebar_rating           423 non-null    float64
 9   IV1                               423 non-null    object 
 10  arrangement                       423 non-null    object 
 11  ecomm_frequence                   423 non-null    object 
 12  trial_i

### Save data to csv

In [129]:
df3.to_csv("qualtrics_data_cleaned.csv", index=False)